In [71]:
#%pip install xgboost

In [72]:
#%pip install wordcloud

In [73]:
#%pip install gensim

In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

In [75]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [76]:
df = train[['facts', 'first_party_winner']]
df_target = df[['first_party_winner']]
df_nlp = df[['facts']]
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True)

dfTest = pd.DataFrame(test['facts'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)



In [77]:
corpus = df_nlp1['facts']
# print(corpus.str.cat(sep=" ")) # 인덱스의 요소들 서로 잇기
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))
lst_tokens

['On',
 'June',
 '27',
 ',',
 '1962',
 ',',
 'Phil',
 'St.',
 'Amant',
 ',',
 'a',
 'candidate',
 'for',
 'public',
 'office',
 ',',
 'made',
 'a',
 'television',
 'speech',
 'in',
 'Baton',
 'Rouge',
 ',',
 'Louisiana',
 '.',
 'During',
 'this',
 'speech',
 ',',
 'St.',
 'Amant',
 'accused',
 'his',
 'political',
 'opponent',
 'of',
 'being',
 'a',
 'Communist',
 'and',
 'of',
 'being',
 'involved',
 'in',
 'criminal',
 'activities',
 'with',
 'the',
 'head',
 'of',
 'the',
 'local',
 'Teamsters',
 'Union',
 '.',
 'Finally',
 ',',
 'St.',
 'Amant',
 'implicated',
 'Herman',
 'Thompson',
 ',',
 'an',
 'East',
 'Baton',
 'Rouge',
 'deputy',
 'sheriff',
 ',',
 'in',
 'a',
 'scheme',
 'to',
 'move',
 'money',
 'between',
 'the',
 'Teamsters',
 'Union',
 'and',
 'St.',
 'Amant',
 '’',
 's',
 'political',
 'opponent',
 '.',
 'Thompson',
 'successfully',
 'sued',
 'St.',
 'Amant',
 'for',
 'defamation',
 '.',
 'Louisiana',
 '’',
 's',
 'First',
 'Circuit',
 'Court',
 'of',
 'Appeals',
 'reve

In [78]:
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()

In [79]:
lst_stopwords = nltk.corpus.stopwords.words("english")

In [80]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [81]:
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

In [82]:
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')

In [83]:
vectorizer = CountVectorizer()

In [84]:
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')
df_nlp2

,facts,facts_clean,first_party_winner
0,"On June 27, 1962, Phil St. Amant, a candidate ...",june 27 1962 phil st amant candidate public of...,1
1,Ramon Nelson was riding his bike when he suffe...,ramon nelson riding bike suffered lethal blow ...,0
2,An Alabama state court convicted Billy Joe Mag...,alabama state court convicted billy joe magwoo...,1
3,Victor Linkletter was convicted in state court...,victor linkletter convicted state court eviden...,0
4,"On April 24, 1953 in Selma, Alabama, an intrud...",april 24 1953 selma alabama intruder broke apa...,1
...,...,...,...
2473,Congress amended the Clean Air Act through the...,congress amended clean air act energy policy a...,1
2474,"Alliance Bond Fund, Inc., an investment fund, ...",alliance bond fund inc investment fund purchas...,1
2475,"In 1992, the District Court sentenced Manuel D...",1992 district court sentenced manuel peguero 2...,0
2476,"On March 8, 1996, Enrico St. Cyr, a lawful per...",march 8 1996 enrico st cyr lawful permanent re...,0


In [85]:
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

In [86]:
# X_train = df_nlp2['facts_clean']
# y_train = df_nlp2['first_party_winner']

# X_test = dfTest['facts_clean']
dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
# y_test = dfTest['first_party_winner']

In [87]:
xfeatures = df_nlp2['facts_clean']
ylabel = df_nlp2['first_party_winner']

In [88]:
X_train, X_test, y_train, y_test = train_test_split(xfeatures,ylabel, test_size=0.25)

In [89]:
pipe = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(solver='liblinear'))])

In [90]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr', LogisticRegression(solver='liblinear'))])

In [91]:
pipe1= Pipeline(steps=[('cv',CountVectorizer()),('rf',RandomForestClassifier())])

In [92]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('rf', RandomForestClassifier())])

In [93]:
pipe1.score(X_train,y_train)

1.0

In [94]:
pipe1.score(X_test,y_test)

0.6935483870967742

In [95]:
pipe2= Pipeline(steps=[('cv',CountVectorizer()),('rf',KNeighborsClassifier(n_neighbors=3))])

In [96]:
pipe2.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('rf', KNeighborsClassifier(n_neighbors=3))])

In [97]:
pipe2.score(X_train,y_train)

0.8697524219590959

In [98]:
pipe2.score(X_test,y_test)

0.6661290322580645

In [99]:
vectorize=CountVectorizer()

In [100]:
count_matrix = vectorize.fit_transform(df_nlp2['facts_clean'])
count_array = count_matrix.toarray()
data_final = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
data_final

,011119,0125,02,036539,04,041352,041581,0479,05,0511287,...,zj,zone,zoned,zoneofinterests,zoning,zubik,zuni,zurich,zurko,zurkos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')

In [102]:
count_matrix = vectorize.fit_transform(X_test)
count_array = count_matrix.toarray()
data_final_test = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
data_final_test = pd.concat([data_final_test,dfTest["first_party_winner"]],axis=1,join='inner')
data_final_test

,011119,04,041581,0479,05,07394,10,100,1000,10000,...,zita,zivotofsky,zivotofskys,zone,zoned,zoneofinterests,zoning,zuni,zurich,first_party_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
# X_train = data_final.drop(columns=['first_party_winner'])
# y_train = data_final['first_party_winner']

X_test = data_final_test.drop(columns=['first_party_winner'])
y_test = data_final_test['first_party_winner']

In [127]:
# X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=10)

In [128]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_final = scaler.fit_transform(data_final)

In [129]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=200, random_state=0)
lda_data = lda.fit_transform(X_train)

In [130]:
lda_data_train = pd.DataFrame(data=lda_data)

In [131]:
lda_data_test = pd.DataFrame(data=lda.transform(X_test))

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- 0125
- 02
- 036539
- 041352
- 0511287
- ...


In [ ]:
rand=RandomForestClassifier(max_depth= 8, max_features = 100, min_samples_leaf = 2, n_estimators = 200)

In [ ]:
rand.fit(lda_data_train,y_train)

RandomForestClassifier(max_depth=8, max_features=100, min_samples_leaf=2,
                       n_estimators=200)

In [ ]:
rand.score(lda_data_train,y_train)

0.7093425605536332

In [ ]:
rand.score(lda_data_test,y_test)

0.6397849462365591

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
y_pred1 = rand.predict(lda_data_test)
f1_score(y_test, y_pred1)

0.7766666666666667

In [ ]:
model = XGBClassifier()
model.fit(lda_data_train, y_train)
y_pred = model.predict(lda_data_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(y_test, y_pred1)


ValueError: Found input variables with inconsistent numbers of samples: [620, 744]

In [ ]:
knn=KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(lda_data_train,y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.score(lda_data_train,y_train)

0.7249134948096886

In [ ]:
knn.score(lda_data_test,y_test)

0.6075268817204301

In [ ]:
submit = pd.read_csv('C:/data/project/sample_submission.csv')
predcsv = pd.DataFrame(predictions,columns=['first_party_winner'])
# submit['first_party_winner'] = predictions
# predcsv
submit['first_party_winner'] = predcsv
submit.to_csv('./baseline_submit.csv', index=False)
print('Done')

Done
